In [2]:
import gzip


def unzip(input, output):

    input_file = input
    output_file = output

    with gzip.open(input_file, "rb") as f_in:
        with open(output_file, "wb") as f_out:

            f_out.write(f_in.read())

    print("file unzipped and save at", output_file)


SyntaxError: invalid syntax. Perhaps you forgot a comma? (1750564590.py, line 16)

In [50]:

import pandas as pd



# read matrix
with open(
    "/data/LSY/z_preparing_and_parts/commontrait_Schizophrenia/GSE41169_series_matrix.txt", "r"
) as file:
    lines = file.readlines()

# find metadata
first_empty_line_index = lines.index("\n")
table_begin_index = next(
(
        i
        for i, line in enumerate(lines)
        if line.startswith("!series_matrix_table_begin")
    ),
    None,
)
end_index = table_begin_index if table_begin_index is not None else len(lines)
content_before_table = lines[first_empty_line_index:end_index]


# read metadata
metadata_lines = [
    line.strip()[1:].split("\t")
    for line in content_before_table
    if line.startswith("!")
]
metadata_df = pd.DataFrame(metadata_lines).T


metadata_df.columns = metadata_df.iloc[0]
metadata_df = metadata_df.drop(0, axis=0)

metadata_df.dropna(inplace=True)


In [63]:
print(metadata_df.iloc[:,16])


1     "diseasestatus (1=control, 2=scz patient): 2"
2     "diseasestatus (1=control, 2=scz patient): 2"
3     "diseasestatus (1=control, 2=scz patient): 2"
4     "diseasestatus (1=control, 2=scz patient): 2"
5     "diseasestatus (1=control, 2=scz patient): 2"
                          ...                      
91    "diseasestatus (1=control, 2=scz patient): 2"
92    "diseasestatus (1=control, 2=scz patient): 2"
93    "diseasestatus (1=control, 2=scz patient): 1"
94    "diseasestatus (1=control, 2=scz patient): 1"
95    "diseasestatus (1=control, 2=scz patient): 2"
Name: Sample_characteristics_ch1, Length: 95, dtype: object


In [70]:
# 导入 re 模块
import re

# 应用函数来确定状态
metadata_df["status"] = metadata_df.iloc[:, 16].apply(
    lambda x: 1 if int(re.findall(r'\d+', x.split(":")[1])[0]) == 2 else 0 if int(re.findall(r'\d+', x.split(":")[1])[0]) == 1 else None
)

# 设置新的索引
metadata_df = metadata_df.set_index("Sample_geo_accession")


In [15]:
print(metadata_df)

0                                                          Sample_title  \
Sample_geo_accession                                                      
"GSM1905080"          "genomic DNA from uninvolved psoriatic skin sa...   
"GSM1905081"          "genomic DNA from uninvolved psoriatic skin sa...   
"GSM1905082"          "genomic DNA from uninvolved psoriatic skin sa...   
"GSM1905083"          "genomic DNA from uninvolved psoriatic skin sa...   
"GSM1905084"          "genomic DNA from uninvolved psoriatic skin sa...   
...                                                                 ...   
"GSM1905301"                 "genomic DNA from  normal skin sample 187"   
"GSM1905302"                 "genomic DNA from  normal skin sample 188"   
"GSM1905303"                 "genomic DNA from  normal skin sample 191"   
"GSM1905304"                       "genomic DNA from  replicates UC185"   
"GSM1905305"                       "genomic DNA from  replicates UC188"   

0                       

In [72]:

metadata_df = metadata_df["status"].apply(pd.to_numeric, errors="coerce")

In [73]:
metadata_df

Sample_geo_accession
"GSM1009660"    1
"GSM1009661"    1
"GSM1009662"    1
"GSM1009663"    1
"GSM1009664"    1
               ..
"GSM1009890"    1
"GSM1009891"    1
"GSM1009892"    0
"GSM1009893"    0
"GSM1009894"    1
Name: status, Length: 95, dtype: int64

In [69]:


# read data
data_lines = [line.strip().split() for line in lines if not line.startswith("!")]
data_df = pd.DataFrame(data_lines).T

data_df.columns = data_df.iloc[0]
data_df = data_df.drop(0, axis=0)

data_df = data_df.set_index('"ID_REF"')
data_df = data_df.iloc[:, 1:]
data_df.columns = [col.strip('"') for col in data_df.columns]

data_df = data_df.apply(pd.to_numeric, errors="coerce")





In [74]:
print(data_df)

              cg00000029  cg00000108  cg00000109  cg00000165  cg00000236  \
"ID_REF"                                                                   
"GSM1009660"    0.454931    0.907883    0.862772    0.160446    0.728180   
"GSM1009661"    0.410287    0.915894    0.881000    0.137600    0.736708   
"GSM1009662"    0.375603    0.939018    0.889573    0.188573    0.601186   
"GSM1009663"    0.487200    0.933975    0.852267    0.112618    0.659397   
"GSM1009664"    0.380866    0.939008    0.844250    0.119975    0.669548   
...                  ...         ...         ...         ...         ...   
"GSM1009890"    0.447089    0.935502    0.865687    0.174138    0.671628   
"GSM1009891"    0.386122    0.931684    0.894564    0.149007    0.632087   
"GSM1009892"    0.462085    0.925136    0.884385    0.152924    0.751258   
"GSM1009893"    0.404526    0.934462    0.870986    0.129513    0.678998   
"GSM1009894"    0.398361    0.948828    0.884202    0.147505    0.667643   

           

In [75]:

# merge metadata and data
data_df["status"] = metadata_df
# drop other disease sample
data_df.dropna(subset=["status"], inplace=True)

In [76]:
data1=data_df

In [22]:
print(data1)

              cg00000029  cg00000108  cg00000109  cg00000165  cg00000236  \
"ID_REF"                                                                   
"GSM1905080"     0.29259     0.91727     0.83746     0.33091     0.86586   
"GSM1905081"     0.38943     0.95521     0.86586     0.30256     0.86530   
"GSM1905082"     0.42140     0.93309     0.84979     0.36217     0.90110   
"GSM1905083"     0.30409     0.92671     0.78840     0.30588     0.87750   
"GSM1905084"     0.33408     0.94815     0.87272     0.30801     0.88944   
...                  ...         ...         ...         ...         ...   
"GSM1905301"     0.26916     0.90001     0.82631     0.28624     0.81995   
"GSM1905302"     0.24330     0.90992     0.84766     0.24782     0.83379   
"GSM1905303"     0.42036     0.93922     0.84657     0.32746     0.87749   
"GSM1905304"     0.30455     0.92300     0.77559     0.23925     0.78473   
"GSM1905305"     0.25864     0.89631     0.79849     0.27985     0.81216   

           

In [96]:

# read matrix
with open(
    "/data/LSY/z_preparing_and_parts/commontrait_Schizophrenia/GSE61107_series_matrix.txt", "r"
) as file:
    lines = file.readlines()

# find metadata
first_empty_line_index = lines.index("\n")
table_begin_index = next(
(
        i
        for i, line in enumerate(lines)
        if line.startswith("!series_matrix_table_begin")
    ),
    None,
)
end_index = table_begin_index if table_begin_index is not None else len(lines)
content_before_table = lines[first_empty_line_index:end_index]


# read metadata
metadata_lines = [
    line.strip()[1:].split("\t")
    for line in content_before_table
    if line.startswith("!")
]
metadata_df = pd.DataFrame(metadata_lines).T


metadata_df.columns = metadata_df.iloc[0]
metadata_df = metadata_df.drop(0, axis=0)

metadata_df.dropna(inplace=True)


In [97]:
print(metadata_df.iloc[:,9].unique)


<bound method Series.unique of 1     "disease status (1=control, 2=scz patient): 2"
2     "disease status (1=control, 2=scz patient): 2"
3     "disease status (1=control, 2=scz patient): 2"
4     "disease status (1=control, 2=scz patient): 2"
5     "disease status (1=control, 2=scz patient): 2"
6     "disease status (1=control, 2=scz patient): 2"
7     "disease status (1=control, 2=scz patient): 2"
8     "disease status (1=control, 2=scz patient): 2"
9     "disease status (1=control, 2=scz patient): 2"
10    "disease status (1=control, 2=scz patient): 2"
11    "disease status (1=control, 2=scz patient): 2"
12    "disease status (1=control, 2=scz patient): 2"
13    "disease status (1=control, 2=scz patient): 2"
14    "disease status (1=control, 2=scz patient): 2"
15    "disease status (1=control, 2=scz patient): 2"
16    "disease status (1=control, 2=scz patient): 2"
17    "disease status (1=control, 2=scz patient): 2"
18    "disease status (1=control, 2=scz patient): 2"
19    "disease 

In [98]:

# 导入 re 模块
import re

# 应用函数来确定状态
metadata_df["status"] = metadata_df.iloc[:, 9].apply(
    lambda x: 1 if int(re.findall(r'\d+', x.split(":")[1])[0]) == 2 else 0 if int(re.findall(r'\d+', x.split(":")[1])[0]) == 1 else None
)
metadata_df = metadata_df.set_index("Sample_geo_accession")

In [99]:

metadata_df = metadata_df["status"].apply(pd.to_numeric, errors="coerce")

In [100]:
print(metadata_df)

Sample_geo_accession
"GSM1496797"    1
"GSM1496798"    1
"GSM1496799"    1
"GSM1496800"    1
"GSM1496801"    1
"GSM1496802"    1
"GSM1496803"    1
"GSM1496804"    1
"GSM1496805"    1
"GSM1496806"    1
"GSM1496807"    1
"GSM1496808"    1
"GSM1496809"    1
"GSM1496810"    1
"GSM1496811"    1
"GSM1496812"    1
"GSM1496813"    1
"GSM1496814"    1
"GSM1496815"    1
"GSM1496816"    1
"GSM1496817"    1
"GSM1496818"    1
"GSM1496819"    1
"GSM1496820"    0
"GSM1496821"    0
"GSM1496822"    0
"GSM1496823"    0
"GSM1496824"    0
"GSM1496825"    0
"GSM1496826"    0
"GSM1496827"    0
"GSM1496828"    0
"GSM1496829"    0
"GSM1496830"    0
"GSM1496831"    0
"GSM1496832"    0
"GSM1496833"    0
"GSM1496834"    0
"GSM1496835"    0
"GSM1496836"    0
"GSM1496837"    0
"GSM1496838"    0
"GSM1496839"    0
"GSM1496840"    0
"GSM1496841"    0
"GSM1496842"    0
"GSM1496843"    0
"GSM1496844"    1
Name: status, dtype: int64


In [101]:


# read data
data_lines = [line.strip().split() for line in lines if not line.startswith("!")]
data_df = pd.DataFrame(data_lines).T

data_df.columns = data_df.iloc[0]
data_df = data_df.drop(0, axis=0)

data_df = data_df.set_index('"ID_REF"')
data_df = data_df.iloc[:, 1:]
data_df.columns = [col.strip('"') for col in data_df.columns]

data_df = data_df.apply(pd.to_numeric, errors="coerce")





In [54]:

def m_to_beta(M):
    return 1 / (1 + 2 ** (-M))



In [55]:
# data_df = data_df.apply(m_to_beta)

In [102]:
print(data_df)

              cg00000029  cg00000108  cg00000109  cg00000165  cg00000236  \
"ID_REF"                                                                   
"GSM1496797"    0.526911    0.897581    0.803627    0.168616    0.840134   
"GSM1496798"    0.646369    0.901479    0.750000    0.197049    0.766820   
"GSM1496799"    0.578568    0.906096    0.698420    0.240889    0.799262   
"GSM1496800"    0.468181    0.919163    0.750111    0.216744    0.828267   
"GSM1496801"    0.555158    0.861807    0.694657    0.323470    0.794374   
"GSM1496802"    0.713469    0.946697    0.789254    0.306464    0.804540   
"GSM1496803"    0.614417    0.914708    0.718425    0.262868    0.710127   
"GSM1496804"    0.682418    0.862779    0.575000    0.274312    0.727506   
"GSM1496805"    0.520753    0.853170    0.597484    0.264108    0.673339   
"GSM1496806"    0.583135    0.942371    0.711170    0.309216    0.799271   
"GSM1496807"    0.572603    0.917683    0.811912    0.244868    0.828634   
"GSM1496808"

In [103]:

# merge metadata and data
data_df["status"] = metadata_df
# drop other disease sample
data_df.dropna(subset=["status"], inplace=True)

In [104]:
concatenated_data = pd.concat([data1, data_df], ignore_index=False, join='outer')

In [105]:
print(concatenated_data['status'])

"ID_REF"
"GSM1009660"    1
"GSM1009661"    1
"GSM1009662"    1
"GSM1009663"    1
"GSM1009664"    1
               ..
"GSM1496840"    0
"GSM1496841"    0
"GSM1496842"    0
"GSM1496843"    0
"GSM1496844"    1
Name: status, Length: 143, dtype: int64


In [106]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(
    concatenated_data, test_size=0.2, random_state=42
)

# save data
train_data.to_csv(
    "/data/LSY/z_preparing_and_parts/commontrait_Schizophrenia/data_train.txt",
    index=True,
    header=True,
    sep="\t",
)


# save data
test_data.to_csv(
    "/data/LSY/z_preparing_and_parts/commontrait_Schizophrenia/data_test.txt",
    index=True,
    header=True,
    sep="\t",
)
